#Grok9 of Backtesterr
@guzmanwolfrank:github
GhostOkaamiii 


In [1]:
#This Version is All Dark and has correct HTML Output file, correct dates to load! 

#Needs: Strategy Code, and run through the logic and make sure it all makes sense!! 
# 
# 
# runner_enhanced_v4.py – AlgoHaus Backtester v4.0
# Created by GhostOkaamiii aka Wolfrank Guzman
# ENHANCED WITH RISK MANAGEMENT, MARGIN CALCULATIONS, AND ADVANCED REPORTING
# FIXED: Duplicate datetime labels issue
import tkinter as tk
from tkinter import ttk, scrolledtext, messagebox
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import webbrowser
import os
import tempfile
import inspect
import pathlib
import threading
import queue
import json
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
# ======================================================================
# 1. MARGIN AND PIP CALCULATIONS
# ======================================================================
class ForexCalculator:
    """Handle all forex calculations including margins, pip values, and position sizing"""
   
    # Leverage options
    LEVERAGE_OPTIONS = [1, 10, 20, 30, 50, 100, 200, 500]
   
    # Standard pip values (for 1 standard lot = 100,000 units)
    PIP_VALUES = {
        'EUR/USD': 0.0001, 'GBP/USD': 0.0001, 'USD/JPY': 0.01,
        'USD/CHF': 0.0001, 'USD/CAD': 0.0001, 'AUD/USD': 0.0001,
        'NZD/USD': 0.0001, 'EUR/JPY': 0.01, 'GBP/JPY': 0.01,
        'AUD/JPY': 0.01, 'EUR/GBP': 0.0001, 'EUR/CHF': 0.0001,
        'GBP/CHF': 0.0001, 'CHF/JPY': 0.01, 'CAD/JPY': 0.01
    }
   
    @staticmethod
    def calculate_pip_value_in_usd(pair, unit_size, current_price):
        """Calculate pip value in USD for given pair and unit size"""
        pip_size = ForexCalculator.PIP_VALUES.get(pair, 0.0001)
       
        if pair.endswith('/USD'):
            # Direct pairs (EUR/USD, GBP/USD, etc.)
            pip_value = pip_size * unit_size
        elif pair.startswith('USD/'):
            # Inverse pairs (USD/JPY, USD/CHF, etc.)
            pip_value = (pip_size / current_price) * unit_size
        else:
            # Cross pairs - simplified calculation
            pip_value = pip_size * unit_size * current_price
       
        return pip_value
   
    @staticmethod
    def calculate_margin_required(pair, unit_size, current_price, leverage):
        """Calculate margin required for a position"""
        if pair.endswith('/USD'):
            position_value = unit_size * current_price
        elif pair.startswith('USD/'):
            position_value = unit_size
        else:
            # Cross pairs
            position_value = unit_size * current_price
       
        margin_required = position_value / leverage
        return margin_required
   
    @staticmethod
    def calculate_position_size(account_balance, risk_percent, stop_loss_pips, pair, current_price):
        """Calculate optimal position size based on risk management"""
        risk_amount = account_balance * (risk_percent / 100)
        pip_value_per_unit = ForexCalculator.calculate_pip_value_in_usd(pair, 1, current_price)
       
        if pip_value_per_unit > 0:
            position_size = risk_amount / (stop_loss_pips * pip_value_per_unit)
        else:
            position_size = 0
           
        return int(position_size) # Round down to nearest unit
# ======================================================================
# 2. ENHANCED DATA LOADER
# ======================================================================
def get_pair_date_range(pair_name: str, base_folder: pathlib.Path):
    """Get the available date range for a pair without loading all data"""
    try:
        pair_clean = pair_name.replace('/', '_')
        subfolder = base_folder / pair_clean
        csv_files = []
       
        if subfolder.is_dir():
            csv_files = list(subfolder.glob("*.csv"))
       
        if not csv_files:
            csv_files = [f for f in base_folder.glob("*.csv") if pair_clean.lower() in f.name.lower()]
       
        if not csv_files:
            return None, None
       
        csv_path = csv_files[0]
       
        # Quick sample to get date range
        df_dates = pd.read_csv(csv_path, usecols=[0], nrows=1000)
        if df_dates.empty:
            return None, None
           
        col_name = df_dates.columns[0]
        df_dates[col_name] = pd.to_datetime(df_dates[col_name], errors='coerce', utc=True)
        df_dates = df_dates.dropna()
       
        if df_dates.empty:
            return None, None
           
        first_date = df_dates[col_name].min()
       
        # Get last date efficiently
        try:
            file_size = csv_path.stat().st_size
            if file_size > 1000000:
                with open(csv_path, 'rb') as f:
                    f.seek(max(0, file_size - 100000))
                    f.readline()
                    tail_data = f.read().decode('utf-8', errors='ignore')
                   
                from io import StringIO
                df_tail = pd.read_csv(StringIO(tail_data), header=None, usecols=[0])
                df_tail[0] = pd.to_datetime(df_tail[0], errors='coerce', utc=True)
                df_tail = df_tail.dropna()
                last_date = df_tail[0].max() if not df_tail.empty else first_date
            else:
                df_all = pd.read_csv(csv_path, usecols=[0])
                df_all[col_name] = pd.to_datetime(df_all[col_name], errors='coerce', utc=True)
                df_all = df_all.dropna()
                last_date = df_all[col_name].max()
        except:
            last_date = first_date
       
        if pd.isna(first_date) or pd.isna(last_date):
            return None, None
           
        return first_date.date(), last_date.date()
    except Exception as e:
        print(f"Error getting date range for {pair_name}: {e}")
        return None, None
def load_pair_data(pair_name: str, base_folder: pathlib.Path, start_date: datetime, end_date: datetime, timeframe: str):
    """Load and optimize data as DataFrame for fast backtesting"""
    pair_clean = pair_name.replace('/', '_')
    subfolder = base_folder / pair_clean
    csv_files = []
   
    if subfolder.is_dir():
        csv_files = list(subfolder.glob("*.csv"))
   
    if not csv_files:
        csv_files = [f for f in base_folder.glob("*.csv") if pair_clean.lower() in f.name.lower()]
   
    if not csv_files:
        raise FileNotFoundError(f"No CSV found for {pair_name}")
   
    csv_path = csv_files[0]
   
    # Read sample to detect columns
    sample = pd.read_csv(csv_path, nrows=100)
    if sample.empty:
        raise ValueError("CSV is empty")
   
    # Column mapping
    cols_lower = [c.strip().lower() for c in sample.columns]
    col_map = {
        'datetime': ['datetime', 'date', 'time', 'timestamp', 'date_time', 'index', 'tick'],
        'open': ['open', 'o', 'bid', 'ask', 'price'],
        'high': ['high', 'h', 'max'],
        'low': ['low', 'l', 'min'],
        'close': ['close', 'c', 'last', 'price'],
        'volume': ['volume', 'vol', 'v', 'tick_volume', 'size']
    }
   
    rename = {}
    for target, aliases in col_map.items():
        for alias in aliases:
            if any(alias in col for col in cols_lower):
                orig = next(col for col in sample.columns if alias in col.lower())
                rename[orig] = target
                break
        else:
            if target != 'volume':
                raise KeyError(f"Column for '{target}' not found")
   
    # Load data
    dtypes = {}
    for col in rename.keys():
        if rename[col] != 'datetime':
            dtypes[col] = 'float32'
   
    df = pd.read_csv(csv_path, usecols=list(rename.keys()), dtype=dtypes)
    df = df.rename(columns=rename)
   
    if 'volume' not in df.columns:
        df['volume'] = 1000
   
    # Parse dates
    df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce', utc=True)
    df = df.dropna(subset=['datetime'])
    df['datetime'] = df['datetime'].dt.tz_localize(None)
    
    # FIXED: Handle duplicate datetimes before sorting
    df = df.drop_duplicates(subset='datetime', keep='first')
    df = df.sort_values('datetime')
   
    # Get actual range
    actual_start = df['datetime'].min().date()
    actual_end = df['datetime'].max().date()
   
    # Filter to date range
    df = df.set_index('datetime')
    user_start = max(pd.Timestamp(start_date.date()), df.index.min())
    user_end = min(pd.Timestamp(end_date.date()) + pd.Timedelta(hours=23, minutes=59, seconds=59), df.index.max())
    df = df.loc[user_start:user_end].copy()
   
    if df.empty:
        raise ValueError(f"No data in range {start_date.date()} to {end_date.date()}")
   
    # Resample if needed
    if timeframe != '1min':
        rule = {'5min': '5T', '15min': '15T', '1hr': '1H', '1Day': '1D'}.get(timeframe, '1T')
        df = df.resample(rule).agg({
            'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum'
        }).dropna()
   
    # Add daily data WITHOUT LOOK-AHEAD BIAS
    df = df.reset_index()
    df['date'] = df['datetime'].dt.date
   
    # Calculate previous day's values (no look-ahead)
    daily = df.groupby('date').agg({
        'high': 'max', 'low': 'min', 'close': 'last'
    })
    daily.columns = ['day_high', 'day_low', 'day_close']
   
    # SHIFT to avoid look-ahead bias
    daily['prev_high'] = daily['day_high'].shift(1)
    daily['prev_low'] = daily['day_low'].shift(1)
    daily['prev_close'] = daily['day_close'].shift(1)
   
    daily = daily.reset_index()
    df = pd.merge(df, daily[['date', 'prev_high', 'prev_low', 'prev_close']], on='date', how='left')
    df[['prev_high', 'prev_low', 'prev_close']] = df[['prev_high', 'prev_low', 'prev_close']].ffill()
   
    return df, actual_start, actual_end
# ======================================================================
# 3. STRATEGIES WITHOUT LOOK-AHEAD BIAS
# ======================================================================
class TradingStrategies:
    @staticmethod
    def opening_range_strategy(df, sl_pips, tp_pips, pip_value):
        """Opening Range Strategy - NO LOOK-AHEAD BIAS"""
        df = df.copy()
        trades = []
       
        # Group by day for daily opening range
        for date in df['date'].unique():
            day_data = df[df['date'] == date].reset_index(drop=True)
            if len(day_data) < 31: # Need at least 30 min + 1 bar
                continue
           
            # First 30 minutes (assuming 1-min data)
            opening_range = day_data.iloc[:30]
            or_high = opening_range['high'].max()
            or_low = opening_range['low'].min()
            or_mid = (or_high + or_low) / 2
           
            # Entry at bar 31 (after 30-min range)
            if len(day_data) > 30:
                entry_bar = day_data.iloc[30]
                signal = 'BUY' if entry_bar['close'] > or_mid else 'SELL'
               
                trades.append({
                    'datetime': entry_bar['datetime'],
                    'entry_price': entry_bar['close'],
                    'signal': signal,
                    'stop_loss': sl_pips * pip_value,
                    'take_profit': tp_pips * pip_value,
                    'day_data': day_data[31:].reset_index(drop=True) # Only future bars
                })
       
        return trades
    @staticmethod
    def vwap_crossover_strategy(df, sl_pips, tp_pips, pip_value):
        """VWAP Crossover - NO LOOK-AHEAD BIAS"""
        df = df.copy()
       
        # Calculate VWAP properly
        df['tpv'] = df['volume'] * (df['high'] + df['low'] + df['close']) / 3
        df['cumvol'] = df.groupby('date')['volume'].cumsum()
        df['cumtpv'] = df.groupby('date')['tpv'].cumsum()
        df['vwap'] = df['cumtpv'] / df['cumvol']
       
        # Detect crossovers using PREVIOUS bar's VWAP
        df['prev_close'] = df['close'].shift(1)
        df['prev_vwap'] = df['vwap'].shift(1)
       
        # Signal when price crosses VWAP
        df['signal'] = np.where(
            (df['prev_close'] <= df['prev_vwap']) & (df['close'] > df['vwap']), 'BUY',
            np.where((df['prev_close'] >= df['prev_vwap']) & (df['close'] < df['vwap']), 'SELL', None)
        )
       
        # Get trade entries
        entries = df[df['signal'].notna()].copy()
        trades = []
       
        for idx, row in entries.iterrows():
            remaining_data = df[df.index > idx].reset_index(drop=True)
            if len(remaining_data) > 0:
                trades.append({
                    'datetime': row['datetime'],
                    'entry_price': row['close'],
                    'signal': row['signal'],
                    'stop_loss': sl_pips * pip_value,
                    'take_profit': tp_pips * pip_value,
                    'day_data': remaining_data
                })
       
        return trades
    @staticmethod
    def pivot_point_reversal_strategy(df, sl_pips, tp_pips, pip_value):
        """Pivot Point Reversal - NO LOOK-AHEAD BIAS"""
        df = df.copy()
       
        # Calculate pivots using PREVIOUS day's data
        df['pivot'] = (df['prev_high'] + df['prev_low'] + df['prev_close']) / 3
        df['r1'] = 2 * df['pivot'] - df['prev_low']
        df['s1'] = 2 * df['pivot'] - df['prev_high']
       
        # Remove first day (no previous data)
        df = df[df['prev_high'].notna()].copy()
       
        trades = []
        bounce_zone = 5 * pip_value
       
        # Check each bar for pivot bounces
        for i in range(1, len(df)): # Start from 1 to use previous bar
            current = df.iloc[i]
           
            # Buy signal at support
            if abs(current['close'] - current['s1']) < bounce_zone:
                remaining = df.iloc[i+1:].reset_index(drop=True)
                if len(remaining) > 0:
                    trades.append({
                        'datetime': current['datetime'],
                        'entry_price': current['close'],
                        'signal': 'BUY',
                        'stop_loss': sl_pips * pip_value,
                        'take_profit': tp_pips * pip_value,
                        'day_data': remaining
                    })
           
            # Sell signal at resistance
            elif abs(current['close'] - current['r1']) < bounce_zone:
                remaining = df.iloc[i+1:].reset_index(drop=True)
                if len(remaining) > 0:
                    trades.append({
                        'datetime': current['datetime'],
                        'entry_price': current['close'],
                        'signal': 'SELL',
                        'stop_loss': sl_pips * pip_value,
                        'take_profit': tp_pips * pip_value,
                        'day_data': remaining
                    })
       
        return trades
# ======================================================================
# 4. ENHANCED BACKTEST ENGINE WITH PROPER CALCULATIONS
# ======================================================================
class EnhancedBacktester:
    def __init__(self, df, initial_balance=10000, unit_size=10000, pip_value=0.0001, leverage=50):
        self.df = df
        self.initial_balance = initial_balance
        self.unit_size = unit_size
        self.pip_value = pip_value
        self.leverage = leverage
        self.results = None
        self.balance_history = []
    def run_backtest(self, strategy_func, sl_pips, tp_pips, pair_name):
        """Run backtest with proper margin and P&L calculations"""
        trades = strategy_func(self.df, sl_pips, tp_pips, self.pip_value)
        if not trades:
            self.results = pd.DataFrame()
            return "No trades generated.", {}
        results = []
        current_balance = self.initial_balance
        trade_number = 1
       
        for t in trades:
            entry_price = t['entry_price']
            signal = t['signal']
            bars = t['day_data']
           
            # Calculate margin required
            margin_required = ForexCalculator.calculate_margin_required(
                pair_name, self.unit_size, entry_price, self.leverage
            )
           
            # Skip trade if insufficient margin
            if margin_required > current_balance * 0.5: # Max 50% margin usage
                continue
           
            # Calculate pip value for this trade
            pip_value_usd = ForexCalculator.calculate_pip_value_in_usd(
                pair_name, self.unit_size, entry_price
            )
           
            # Set stop loss and take profit levels
            if signal == 'BUY':
                stop_level = entry_price - (sl_pips * self.pip_value)
                take_level = entry_price + (tp_pips * self.pip_value)
            else:
                stop_level = entry_price + (sl_pips * self.pip_value)
                take_level = entry_price - (tp_pips * self.pip_value)
           
            # Find exit point
            exit_idx = len(bars) - 1
            exit_reason = 'Timeout'
            exit_price = bars.iloc[-1]['close'] if len(bars) > 0 else entry_price
           
            for i, bar in bars.iterrows():
                if signal == 'BUY':
                    if bar['low'] <= stop_level:
                        exit_idx = i
                        exit_price = stop_level
                        exit_reason = 'SL'
                        break
                    elif bar['high'] >= take_level:
                        exit_idx = i
                        exit_price = take_level
                        exit_reason = 'TP'
                        break
                else: # SELL
                    if bar['high'] >= stop_level:
                        exit_idx = i
                        exit_price = stop_level
                        exit_reason = 'SL'
                        break
                    elif bar['low'] <= take_level:
                        exit_idx = i
                        exit_price = take_level
                        exit_reason = 'TP'
                        break
           
            # Calculate P&L
            if signal == 'BUY':
                pips_pnl = (exit_price - entry_price) / self.pip_value
            else:
                pips_pnl = (entry_price - exit_price) / self.pip_value
           
            monetary_pnl = pips_pnl * pip_value_usd
           
            # Calculate time in trade
            entry_time = t['datetime']
            if exit_idx < len(bars):
                exit_time = bars.iloc[exit_idx]['datetime']
            else:
                exit_time = bars.iloc[-1]['datetime'] if len(bars) > 0 else entry_time
           
            time_in_trade = exit_time - entry_time
            hours_in_trade = time_in_trade.total_seconds() / 3600
           
            # Update balance
            current_balance += monetary_pnl
           
            results.append({
                'trade_number': f"{trade_number:05d}",
                'entry_time': entry_time,
                'exit_time': exit_time,
                'time_in_trade_hours': round(hours_in_trade, 2),
                'signal': signal,
                'entry_price': entry_price,
                'exit_price': exit_price,
                'exit_reason': exit_reason,
                'pips_pnl': round(pips_pnl, 2),
                'monetary_pnl': round(monetary_pnl, 2),
                'unit_size': self.unit_size,
                'margin_used': round(margin_required, 2),
                'balance': round(current_balance, 2),
                'pip_value_usd': round(pip_value_usd, 4)
            })
           
            trade_number += 1
            self.balance_history.append(current_balance)
       
        self.results = pd.DataFrame(results)
       
        # Calculate metrics
        if not self.results.empty:
            total_pnl = self.results['monetary_pnl'].sum()
            total_pips = self.results['pips_pnl'].sum()
            win_rate = (self.results['pips_pnl'] > 0).mean() * 100
           
            summary = (f"TRADES: {len(self.results)}\n"
                      f"WIN RATE: {win_rate:.1f}%\n"
                      f"P&L: ${total_pnl:,.2f}\n"
                      f"PIPS: {total_pips:,.1f}\n"
                      f"FINAL BALANCE: ${current_balance:,.2f}")
           
            metrics = QuantStatsMetrics.calculate_all_metrics(
                self.results, self.initial_balance
            )
        else:
            summary = "No trades executed"
            metrics = {}
       
        return summary, metrics
# ======================================================================
# 5. QUANTSTATS-STYLE METRICS
# ======================================================================
class QuantStatsMetrics:
    @staticmethod
    def calculate_all_metrics(trades_df, initial_balance):
        """Calculate comprehensive QuantStats-style metrics"""
        if trades_df.empty:
            return {}
       
        # Basic metrics
        total_trades = len(trades_df)
        winning_trades = (trades_df['monetary_pnl'] > 0).sum()
        losing_trades = (trades_df['monetary_pnl'] < 0).sum()
        win_rate = (winning_trades / total_trades * 100) if total_trades > 0 else 0
       
        # P&L metrics
        total_pnl = trades_df['monetary_pnl'].sum()
        total_pips = trades_df['pips_pnl'].sum()
        avg_win = trades_df[trades_df['monetary_pnl'] > 0]['monetary_pnl'].mean() if winning_trades > 0 else 0
        avg_loss = abs(trades_df[trades_df['monetary_pnl'] < 0]['monetary_pnl'].mean()) if losing_trades > 0 else 0
        avg_win_pips = trades_df[trades_df['pips_pnl'] > 0]['pips_pnl'].mean() if winning_trades > 0 else 0
        avg_loss_pips = abs(trades_df[trades_df['pips_pnl'] < 0]['pips_pnl'].mean()) if losing_trades > 0 else 0
       
        # Best and worst trades
        best_trade = trades_df['monetary_pnl'].max()
        worst_trade = trades_df['monetary_pnl'].min()
        best_trade_pips = trades_df['pips_pnl'].max()
        worst_trade_pips = trades_df['pips_pnl'].min()
       
        # Profit factor
        gross_profit = trades_df[trades_df['monetary_pnl'] > 0]['monetary_pnl'].sum()
        gross_loss = abs(trades_df[trades_df['monetary_pnl'] < 0]['monetary_pnl'].sum())
        profit_factor = gross_profit / gross_loss if gross_loss > 0 else float('inf')
       
        # Returns
        final_balance = trades_df['balance'].iloc[-1] if not trades_df.empty else initial_balance
        total_return = ((final_balance - initial_balance) / initial_balance) * 100
       
        # Create equity curve
        equity_curve = initial_balance + trades_df['monetary_pnl'].cumsum()
       
        # Drawdown calculations
        cummax = equity_curve.expanding().max()
        drawdown = (equity_curve - cummax) / cummax
        max_drawdown_pct = drawdown.min() * 100
       
        # Time-based metrics
        start_date = trades_df['entry_time'].min()
        end_date = trades_df['exit_time'].max()
        trading_days = (end_date - start_date).days if pd.notna(start_date) and pd.notna(end_date) else 1
       
        # Annualized return
        years = trading_days / 365 if trading_days > 0 else 1
        annualized_return = ((final_balance / initial_balance) ** (1/years) - 1) * 100 if years > 0 else 0
       
        # Sharpe Ratio (simplified - using daily returns)
        if len(trades_df) > 1:
            daily_returns = trades_df.groupby(trades_df['entry_time'].dt.date)['monetary_pnl'].sum()
            daily_returns_pct = daily_returns / initial_balance
            sharpe = (daily_returns_pct.mean() * 252) / (daily_returns_pct.std() * np.sqrt(252)) if daily_returns_pct.std() > 0 else 0
        else:
            sharpe = 0
       
        # Sortino Ratio (using downside returns only)
        if len(trades_df) > 1:
            negative_returns = daily_returns_pct[daily_returns_pct < 0]
            downside_std = negative_returns.std() if len(negative_returns) > 0 else 0
            sortino = (daily_returns_pct.mean() * 252) / (downside_std * np.sqrt(252)) if downside_std > 0 else 0
        else:
            sortino = 0
       
        # Average time in trade
        avg_time_in_trade = trades_df['time_in_trade_hours'].mean() if 'time_in_trade_hours' in trades_df else 0
       
        # Margin metrics
        avg_margin = trades_df['margin_used'].mean() if 'margin_used' in trades_df else 0
       
        # Average trade PnL
        avg_trade_pnl = total_pnl / total_trades if total_trades > 0 else 0
       
        return {
            # Trade Statistics
            'total_trades': total_trades,
            'winning_trades': winning_trades,
            'losing_trades': losing_trades,
            'win_rate_%': round(win_rate, 2),
           
            # P&L Metrics
            'total_pnl_$': round(total_pnl, 2),
            'total_pips': round(total_pips, 1),
            'avg_win_$': round(avg_win, 2),
            'avg_loss_$': round(avg_loss, 2),
            'avg_win_pips': round(avg_win_pips, 1),
            'avg_loss_pips': round(avg_loss_pips, 1),
            'avg_trade_pnl_$': round(avg_trade_pnl, 2),
           
            # Best/Worst
            'best_trade_$': round(best_trade, 2),
            'worst_trade_$': round(worst_trade, 2),
            'best_trade_pips': round(best_trade_pips, 1),
            'worst_trade_pips': round(worst_trade_pips, 1),
           
            # Risk Metrics
            'profit_factor': round(profit_factor, 2),
            'max_drawdown_%': round(max_drawdown_pct, 2),
            'sharpe_ratio': round(sharpe, 2),
            'sortino_ratio': round(sortino, 2),
           
            # Return Metrics
            'total_return_%': round(total_return, 2),
            'annualized_return_%': round(annualized_return, 2),
           
            # Other Metrics
            'avg_time_in_trade_hours': round(avg_time_in_trade, 2),
            'avg_margin_used_$': round(avg_margin, 2),
            'final_balance_$': round(final_balance, 2)
        }
        
# ======================================================================
# 6. ADVANCED HTML REPORT GENERATOR
# ======================================================================
class AdvancedHTMLReportGenerator:
    @staticmethod
    def generate_dashboard_report(metrics, trades_df, strategy, timeframe, pair, initial_balance, unit_size, df=None):
        """Generate comprehensive HTML report with dashboard elements"""
       
        # Create timestamp for filename
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
       
        # Get strategy source code
        strategy_code = AdvancedHTMLReportGenerator.get_strategy_code(strategy)
       
        # Generate the comparison chart first (for top placement)
        comparison_chart_html = AdvancedHTMLReportGenerator.generate_comparison_chart(trades_df, initial_balance, df)
       
        # Generate other plots
        plots_html = AdvancedHTMLReportGenerator.generate_plots(trades_df, initial_balance, df)
       
        # AI-style analysis text
        analysis = AdvancedHTMLReportGenerator.generate_ai_analysis(metrics, trades_df, df)
       
        html = f"""
        <!DOCTYPE html>
        <html>
        <head>
            <title>AlgoHaus Backtest Report - {pair} {strategy}</title>
            <meta charset="utf-8">
            <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
            <style>
                body {{
                    font-family: 'Consolas', 'Courier New', monospace;
                    background: #000000;
                    color: #ffffff;
                    margin: 0;
                    padding: 0;
                }}
                .header {{
                    background: #000000;
                    color: #ffffff;
                    padding: 20px;
                    border-bottom: 1px solid #333;
                }}
                .header h1 {{
                    margin: 0;
                    font-size: 18px;
                    font-weight: bold;
                    font-family: 'Consolas', monospace;
                }}
                .primary-chart {{
                    background: #0a0a0a;
                    padding: 20px;
                    border-bottom: 2px solid #1976D2;
                }}
                .primary-chart h2 {{
                    color: #1976D2;
                    font-size: 20px;
                    margin: 0 0 20px 0;
                    font-weight: bold;
                    font-family: 'Consolas', monospace;
                }}
                .strategy-code-section {{
                    background: #0a0a0a;
                    padding: 20px;
                    border-bottom: 1px solid #333;
                }}
                .strategy-code-button {{
                    background: #1976D2;
                    color: white;
                    border: none;
                    padding: 10px 20px;
                    font-size: 14px;
                    font-family: 'Consolas', monospace;
                    cursor: pointer;
                    border-radius: 4px;
                    margin-bottom: 15px;
                }}
                .strategy-code-button:hover {{
                    background: #1565C0;
                }}
                .strategy-code-content {{
                    display: none;
                    background: #1a1a1a;
                    border: 1px solid #333;
                    border-radius: 5px;
                    padding: 20px;
                    overflow-x: auto;
                }}
                .strategy-code-content.show {{
                    display: block;
                }}
                .strategy-code-content pre {{
                    margin: 0;
                    color: #e0e0e0;
                    font-family: 'Consolas', 'Courier New', monospace;
                    font-size: 12px;
                    line-height: 1.4;
                    white-space: pre;
                }}
                .code-keyword {{ color: #569CD6; }}
                .code-string {{ color: #CE9178; }}
                .code-comment {{ color: #6A9955; }}
                .code-function {{ color: #DCDCAA; }}
                .code-number {{ color: #B5CEA8; }}
                .metrics-section {{
                    background: #000000;
                    color: #ffffff;
                    padding: 20px;
                    font-family: 'Consolas', monospace;
                    border-bottom: 1px solid #333;
                }}
                .metrics-section h2 {{
                    font-size: 16px;
                    margin: 0 0 15px 0;
                    font-weight: bold;
                }}
                .metrics-table {{
                    font-size: 12px;
                    line-height: 1.4;
                }}
                .trades-section {{
                    background: #0a0a0a;
                    padding: 20px;
                    border-bottom: 1px solid #333;
                }}
                .trades-section h2 {{
                    color: #ffffff;
                    font-size: 16px;
                    margin: 0 0 15px 0;
                    font-weight: bold;
                    font-family: 'Consolas', monospace;
                }}
                table {{
                    width: 100%;
                    border-collapse: collapse;
                    font-family: 'Consolas', monospace;
                    font-size: 12px;
                }}
                th {{
                    background: #1a1a1a;
                    color: #ffffff;
                    padding: 8px 4px;
                    text-align: left;
                    font-weight: normal;
                    border-bottom: 1px solid #333;
                    white-space: nowrap;
                }}
                td {{
                    padding: 6px 4px;
                    border-bottom: 1px solid #111;
                    color: #cccccc;
                    white-space: nowrap;
                }}
                tr:hover {{
                    background: #1a1a1a;
                }}
                .positive {{ color: #00ff00; }}
                .negative {{ color: #ff4444; }}
                .neutral {{ color: #ffff00; }}
               
                /* Enhanced sections with dark theme */
                .analysis-section {{
                    background: linear-gradient(135deg, #1a1a1a 0%, #2d2d2d 100%);
                    padding: 30px;
                    margin: 20px;
                    border-radius: 10px;
                    border: 1px solid #333;
                }}
                .analysis-section h2 {{
                    color: #1976D2;
                    font-size: 18px;
                    border-bottom: 2px solid #1976D2;
                    padding-bottom: 10px;
                    margin-bottom: 20px;
                }}
                .analysis-text {{
                    color: #e0e0e0;
                    line-height: 1.8;
                    font-family: 'Segoe UI', Arial, sans-serif;
                    font-size: 14px;
                }}
                .plot-container {{
                    margin: 20px;
                    padding: 20px;
                    background: #1a1a1a;
                    border-radius: 10px;
                    border: 1px solid #333;
                }}
                .advanced-metrics {{
                    background: #0f0f0f;
                    padding: 20px;
                    margin: 20px;
                    border-radius: 10px;
                    border: 1px solid #333;
                }}
                .metrics-grid {{
                    display: grid;
                    grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
                    gap: 15px;
                    margin-top: 20px;
                }}
                .metric-card {{
                    background: #1a1a1a;
                    padding: 15px;
                    border-radius: 5px;
                    border-left: 3px solid #1976D2;
                }}
                .metric-card h3 {{
                    margin: 0;
                    font-size: 11px;
                    color: #888;
                    text-transform: uppercase;
                    letter-spacing: 1px;
                }}
                .metric-card .value {{
                    font-size: 20px;
                    font-weight: bold;
                    margin-top: 5px;
                }}
                .footer {{
                    text-align: center;
                    padding: 20px;
                    background: #000000;
                    color: #666;
                    font-size: 11px;
                    border-top: 1px solid #333;
                }}
            </style>
            <script>
                function toggleStrategyCode() {{
                    var content = document.getElementById('strategyCodeContent');
                    var button = document.getElementById('strategyCodeButton');
                    if (content.classList.contains('show')) {{
                        content.classList.remove('show');
                        button.textContent = '< > Show Strategy Code';
                    }} else {{
                        content.classList.add('show');
                        button.textContent = '< > Hide Strategy Code';
                    }}
                }}
            </script>
        </head>
        <body>
            <!-- Classic Black Header -->
            <div class="header">
                <h1>{strategy} – {pair} ({timeframe})</h1>
                <p>GhostOkaamiii, @guzmanwolfrank:github</p>
            </div>
           
            <!-- PRIMARY CHART: Strategy vs Underlying Performance (TOP OF PAGE) -->
            <div class="primary-chart">
                <h2>📈 Strategy vs Underlying Performance</h2>
                {comparison_chart_html}
            </div>
           
            <!-- Strategy Code Section -->
            <div class="strategy-code-section">
                <button id="strategyCodeButton" class="strategy-code-button" onclick="toggleStrategyCode()">
                    &lt; &gt; Show Strategy Code
                </button>
                <div id="strategyCodeContent" class="strategy-code-content">
                    <pre>{strategy_code}</pre>
                </div>
            </div>
           
            <!-- Metrics Section - Classic Style -->
            <div class="metrics-section">
                <h2>Metrics</h2>
                <div class="metrics-table">
Total Trades: {metrics.get('total_trades', 0)}<br>
Win Rate: {metrics.get('win_rate_%', 0):.1f}%<br>
Total PnL: ${metrics.get('total_pnl_$', 0):,.0f}<br>
Total Pips: {metrics.get('total_pips', 0):,.1f}<br>
Avg Win USD: ${metrics.get('avg_win_$', 0):,.0f}<br>
Avg Loss USD: ${metrics.get('avg_loss_$', 0):,.0f}<br>
Avg Win Pips: {metrics.get('avg_win_pips', 0):,.0f}<br>
Avg Loss Pips: {metrics.get('avg_loss_pips', 0):,.0f}<br>
Avg Win Usd 10K: ${metrics.get('avg_win_$', 0) * (10000/unit_size):,.0f}<br>
Avg Loss Usd 10K: ${metrics.get('avg_loss_$', 0) * (10000/unit_size):,.0f}<br>
Profit Factor: {metrics.get('profit_factor', 0):.2f}<br>
Max Drawdown Pct: {metrics.get('max_drawdown_%', 0):.1f}%<br>
Annualized Return Pct: {metrics.get('annualized_return_%', 0):.1f}%<br>
Rate Of Return Pct: {metrics.get('total_return_%', 0):.1f}%<br>
Sharpe Ratio: {metrics.get('sharpe_ratio', 0):.2f}<br>
Sortino Ratio: {metrics.get('sortino_ratio', 0):.2f}<br>
Biggest Win: ${metrics.get('best_trade_$', 0):,.0f}<br>
Biggest Loss: ${metrics.get('worst_trade_$', 0):,.0f}<br>
Avg Margin: ${metrics.get('avg_margin_used_$', 0):,.0f}<br>
                </div>
            </div>
           
            <!-- Trades Section - Classic Style -->
            <div class="trades-section">
                <h2>Trades</h2>
                <details open>
                    <summary>Trades Summary</summary>
                    <table>
                        <thead>
                            <tr>
                                <th>strategy</th>
                                <th>entry_time</th>
                                <th>exit_time</th>
                                <th>time_in_trade</th>
                                <th>signal</th>
                                <th>entry_price</th>
                                <th>exit_price</th>
                                <th>exit_reason</th>
                                <th>pips_pnl</th>
                                <th>monetary_pnl</th>
                                <th>units</th>
                                <th>margin</th>
                            </tr>
                        </thead>
                        <tbody>
                            {''.join([f'''
                            <tr>
                                <td>{strategy}</td>
                                <td>{row['entry_time'].strftime('%Y-%m-%d %H:%M:%S')}</td>
                                <td>{row['exit_time'].strftime('%Y-%m-%d %H:%M:%S')}</td>
                                <td>{int(row['time_in_trade_hours'])} days {int((row['time_in_trade_hours'] % 24)):02d}:{int((row['time_in_trade_hours'] * 60) % 60):02d}:00</td>
                                <td>{row['signal']}</td>
                                <td>{row['entry_price']:.5f}</td>
                                <td>{row['exit_price']:.5f}</td>
                                <td>{row['exit_reason']}</td>
                                <td class="{'positive' if row['pips_pnl'] > 0 else 'negative'}">{row['pips_pnl']:.1f}</td>
                                <td class="{'positive' if row['monetary_pnl'] > 0 else 'negative'}">{row['monetary_pnl']:.1f}</td>
                                <td>{row['unit_size']:.1f}</td>
                                <td>{row['margin_used']:.2f}</td>
                            </tr>
                            ''' for _, row in trades_df.head(100).iterrows()])}
                        </tbody>
                    </table>
                </details>
            </div>
           
            <!-- Advanced Metrics Section -->
            <div class="advanced-metrics">
                <h2 style="color: #1976D2; font-size: 18px; margin-bottom: 20px;">📊 Advanced Performance Metrics</h2>
                <div class="metrics-grid">
                    <div class="metric-card">
                        <h3>Total Return</h3>
                        <div class="value {'positive' if metrics.get('total_return_%', 0) > 0 else 'negative'}">{metrics.get('total_return_%', 0):.2f}%</div>
                    </div>
                    <div class="metric-card">
                        <h3>Win Rate</h3>
                        <div class="value {'positive' if metrics.get('win_rate_%', 0) > 50 else 'negative'}">{metrics.get('win_rate_%', 0):.1f}%</div>
                    </div>
                    <div class="metric-card">
                        <h3>Sharpe Ratio</h3>
                        <div class="value {'positive' if metrics.get('sharpe_ratio', 0) > 1 else 'negative' if metrics.get('sharpe_ratio', 0) < 0 else 'neutral'}">{metrics.get('sharpe_ratio', 0):.2f}</div>
                    </div>
                    <div class="metric-card">
                        <h3>Sortino Ratio</h3>
                        <div class="value {'positive' if metrics.get('sortino_ratio', 0) > 1 else 'negative' if metrics.get('sortino_ratio', 0) < 0 else 'neutral'}">{metrics.get('sortino_ratio', 0):.2f}</div>
                    </div>
                    <div class="metric-card">
                        <h3>Max Drawdown</h3>
                        <div class="value negative">{metrics.get('max_drawdown_%', 0):.2f}%</div>
                    </div>
                    <div class="metric-card">
                        <h3>Profit Factor</h3>
                        <div class="value {'positive' if metrics.get('profit_factor', 0) > 1.5 else 'negative' if metrics.get('profit_factor', 0) < 1 else 'neutral'}">{metrics.get('profit_factor', 0):.2f}</div>
                    </div>
                    <div class="metric-card">
                        <h3>Best Trade</h3>
                        <div class="value positive">${metrics.get('best_trade_$', 0):,.0f}</div>
                    </div>
                    <div class="metric-card">
                        <h3>Worst Trade</h3>
                        <div class="value negative">${metrics.get('worst_trade_$', 0):,.0f}</div>
                    </div>
                    <div class="metric-card">
                        <h3>Avg Time in Trade</h3>
                        <div class="value neutral">{metrics.get('avg_time_in_trade_hours', 0):.1f}h</div>
                    </div>
                    <div class="metric-card">
                        <h3>Total Trades</h3>
                        <div class="value neutral">{metrics.get('total_trades', 0)}</div>
                    </div>
                    <div class="metric-card">
                        <h3>Winning Trades</h3>
                        <div class="value positive">{metrics.get('winning_trades', 0)}</div>
                    </div>
                    <div class="metric-card">
                        <h3>Losing Trades</h3>
                        <div class="value negative">{metrics.get('losing_trades', 0)}</div>
                    </div>
                    <div class="metric-card">
                        <h3>Average Trade PnL</h3>
                        <div class="value {'positive' if metrics.get('avg_trade_pnl_$', 0) > 0 else 'negative'}" style="color: white;">${metrics.get('avg_trade_pnl_$', 0):,.2f}</div>
                    </div>
                </div>
            </div>
           
            <!-- AI Analysis Section -->
            <div class="analysis-section">
                <h2>🤖 AI Performance Analysis</h2>
                <div class="analysis-text">
                    {analysis}
                </div>
            </div>
           
            <!-- Interactive Charts -->
            {plots_html}
           
            <!-- Full Trade Details -->
            <div class="trades-section">
                <h2>📝 Complete Trade Log</h2>
                <details>
                    <summary>Complete Trade Log</summary>
                    <table>
                        <thead>
                            <tr>
                                <th>Trade #</th>
                                <th>Entry Time</th>
                                <th>Exit Time</th>
                                <th>Duration</th>
                                <th>Type</th>
                                <th>Entry</th>
                                <th>Exit</th>
                                <th>Reason</th>
                                <th>Pips</th>
                                <th>P&L</th>
                                <th>Balance</th>
                                <th>Units</th>
                                <th>Margin</th>
                            </tr>
                        </thead>
                        <tbody>
                            {''.join([f'''
                            <tr>
                                <td>{row['trade_number']}</td>
                                <td>{row['entry_time'].strftime('%Y-%m-%d %H:%M')}</td>
                                <td>{row['exit_time'].strftime('%Y-%m-%d %H:%M')}</td>
                                <td>{row['time_in_trade_hours']:.1f}h</td>
                                <td>{row['signal']}</td>
                                <td>{row['entry_price']:.5f}</td>
                                <td>{row['exit_price']:.5f}</td>
                                <td>{row['exit_reason']}</td>
                                <td class="{'positive' if row['pips_pnl'] > 0 else 'negative'}">{row['pips_pnl']:.1f}</td>
                                <td class="{'positive' if row['monetary_pnl'] > 0 else 'negative'}">${row['monetary_pnl']:.2f}</td>
                                <td>${row['balance']:.2f}</td>
                                <td>{row['unit_size']:,}</td>
                                <td>${row['margin_used']:.2f}</td>
                            </tr>
                            ''' for _, row in trades_df.iterrows()])}
                        </tbody>
                    </table>
                </details>
            </div>
           
            <div class="footer">
                <p>AlgoHaus Backtester v4.0 | Created by GhostOkaamiii (Wolfrank Guzman)</p>
                <p>Report generated on {datetime.now().strftime("%Y-%m-%d %H:%M:%S")} | File: backtest_{pair.replace('/', '')}_{strategy.replace(' ', '_')}_{timestamp}.html</p>
            </div>
        </body>
        </html>
        """
       
        return html, timestamp
   
    @staticmethod
    def get_strategy_code(strategy_name):
        """Get the source code of the strategy for display"""
        try:
            # Get the strategy function from TradingStrategies class
            if strategy_name == "Opening Range":
                func = TradingStrategies.opening_range_strategy
            elif strategy_name == "VWAP Crossover":
                func = TradingStrategies.vwap_crossover_strategy
            elif strategy_name == "Pivot Reversal":
                func = TradingStrategies.pivot_point_reversal_strategy
            else:
                return "# Strategy code not found"
            
            # Get the source code
            source = inspect.getsource(func)
            
            # Apply basic syntax highlighting
            # Keywords
            keywords = ['def', 'return', 'if', 'else', 'elif', 'for', 'while', 'in', 'import', 
                       'from', 'class', 'try', 'except', 'raise', 'with', 'as', 'lambda',
                       'and', 'or', 'not', 'is', 'None', 'True', 'False', 'break', 'continue']
            
            # HTML escape
            source = source.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;')
            
            # Apply syntax highlighting
            for keyword in keywords:
                source = source.replace(f' {keyword} ', f' <span class="code-keyword">{keyword}</span> ')
                source = source.replace(f' {keyword}(', f' <span class="code-keyword">{keyword}</span>(')
                source = source.replace(f' {keyword}:', f' <span class="code-keyword">{keyword}</span>:')
                source = source.replace(f'({keyword} ', f'(<span class="code-keyword">{keyword}</span> ')
                source = source.replace(f'\n{keyword} ', f'\n<span class="code-keyword">{keyword}</span> ')
                source = source.replace(f'\n{keyword}(', f'\n<span class="code-keyword">{keyword}</span>(')
            
            # Highlight strings (basic)
            import re
            # Single quotes
            source = re.sub(r"'([^']*)'", r"<span class='code-string'>'\1'</span>", source)
            # Double quotes
            source = re.sub(r'"([^"]*)"', r'<span class="code-string">"\1"</span>', source)
            
            # Highlight comments
            lines = source.split('\n')
            highlighted_lines = []
            for line in lines:
                if '#' in line:
                    parts = line.split('#', 1)
                    if len(parts) == 2:
                        line = parts[0] + '<span class="code-comment">#' + parts[1] + '</span>'
                highlighted_lines.append(line)
            source = '\n'.join(highlighted_lines)
            
            # Highlight function names
            source = re.sub(r'\b(def\s+)(\w+)', r'\1<span class="code-function">\2</span>', source)
            
            return source
            
        except Exception as e:
            return f"# Error retrieving strategy code: {str(e)}"
    
    @staticmethod
    def generate_comparison_chart(trades_df, initial_balance, df=None):
        """Generate the comparison chart for top placement"""
        comparison_html = ""
        if df is not None and not df.empty and not trades_df.empty:
            try:
                # Create a copy and ensure no duplicate datetime values
                df_copy = df.copy()
                
                # Remove duplicates if they exist
                if 'datetime' in df_copy.columns:
                    df_copy = df_copy.drop_duplicates(subset=['datetime'], keep='first')
                    df_copy = df_copy.sort_values('datetime')
                
                # Create equity DataFrame
                equity_df = pd.DataFrame({
                    'exit_time': trades_df['exit_time'],
                    'equity': initial_balance + trades_df['monetary_pnl'].cumsum()
                })
                
                # Sort and remove any duplicate exit times
                equity_df = equity_df.drop_duplicates(subset=['exit_time'], keep='last')
                equity_df = equity_df.sort_values('exit_time')
                equity_df = equity_df.set_index('exit_time')
                
                # Instead of reindex which fails with duplicates, use merge_asof
                df_for_merge = pd.DataFrame({
                    'datetime': df_copy['datetime'],
                    'close': df_copy['close']
                }).reset_index(drop=True)
                
                equity_for_merge = equity_df.reset_index()
                equity_for_merge = equity_for_merge.rename(columns={'exit_time': 'datetime'})
                
                # Merge the data using merge_asof for time-based alignment
                merged = pd.merge_asof(
                    df_for_merge.sort_values('datetime'),
                    equity_for_merge.sort_values('datetime'),
                    on='datetime',
                    direction='backward'
                )
                
                # Fill NaN values with initial balance
                merged['equity'] = merged['equity'].fillna(initial_balance)
                
                # Create the comparison plot
                fig = make_subplots(specs=[[{"secondary_y": True}]])
                fig.add_trace(
                    go.Scatter(
                        x=merged['datetime'], 
                        y=merged['equity'], 
                        name="Strategy Equity", 
                        line=dict(color='#1976D2', width=2.5)
                    ),
                    secondary_y=False,
                )
                fig.add_trace(
                    go.Scatter(
                        x=merged['datetime'], 
                        y=merged['close'], 
                        name="Pair Price", 
                        line=dict(color='#FF9900', width=2)
                    ),
                    secondary_y=True,
                )
                fig.update_layout(
                    xaxis_title='Date',
                    yaxis_title='Equity ($)',
                    yaxis2_title='Price',
                    template='plotly_dark',
                    height=500,  # Larger height for primary chart
                    showlegend=True,
                    legend=dict(
                        x=0.02,
                        y=0.98,
                        bgcolor='rgba(0,0,0,0.5)',
                        bordercolor='#333',
                        borderwidth=1
                    )
                )
                comparison_html = fig.to_html(include_plotlyjs=False, div_id="comparison-plot")
            except Exception as e:
                print(f"Warning: Could not create comparison chart: {e}")
                comparison_html = "<p style='color: #666; text-align: center; padding: 40px;'>Unable to generate comparison chart</p>"
        else:
            comparison_html = "<p style='color: #666; text-align: center; padding: 40px;'>Insufficient data for comparison chart</p>"
        
        return comparison_html
   
    @staticmethod
    def generate_plots(trades_df, initial_balance, df=None):
        """Generate interactive plots for the dashboard (excluding comparison chart)"""
        if trades_df.empty:
            return ""
       
        # Equity Curve
        equity = [initial_balance]
        for pnl in trades_df['monetary_pnl']:
            equity.append(equity[-1] + pnl)
       
        dates = [trades_df['entry_time'].iloc[0]] + list(trades_df['exit_time'])
       
        equity_plot = go.Figure()
        equity_plot.add_trace(go.Scatter(
            x=dates, y=equity[:-1],
            mode='lines+markers',
            name='Equity',
            line=dict(color='#1976D2', width=2),
            marker=dict(size=5)
        ))
       
        equity_plot.update_layout(
            title='Equity Curve',
            xaxis_title='Date',
            yaxis_title='Balance ($)',
            template='plotly_dark',
            height=400
        )
       
        equity_html = equity_plot.to_html(include_plotlyjs=False, div_id="equity-plot")
       
        # P&L Distribution Histogram
        pnl_hist = go.Figure()
        pnl_hist.add_trace(go.Histogram(
            x=trades_df['monetary_pnl'],
            nbinsx=30,
            marker_color='#1976D2',
            opacity=0.75
        ))
       
        pnl_hist.update_layout(
            title='P&L Distribution',
            xaxis_title='P&L ($)',
            yaxis_title='Frequency',
            template='plotly_dark',
            height=400
        )
       
        pnl_html = pnl_hist.to_html(include_plotlyjs=False, div_id="pnl-plot")
       
        # Win/Loss Heatmap by Hour and Day
        heatmap_html = ""
        heatmap_description = ""
        if 'entry_time' in trades_df.columns:
            trades_df_copy = trades_df.copy()  # Work with a copy to avoid modifying original
            trades_df_copy['hour'] = pd.to_datetime(trades_df_copy['entry_time']).dt.hour
            trades_df_copy['weekday'] = pd.to_datetime(trades_df_copy['entry_time']).dt.dayofweek
           
            # Create pivot table
            heatmap_data = trades_df_copy.pivot_table(
                values='monetary_pnl',
                index='hour',
                columns='weekday',
                aggfunc='mean',
                fill_value=0
            )
           
            if not heatmap_data.empty:
                heatmap = go.Figure(data=go.Heatmap(
                    z=heatmap_data.values,
                    x=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'][:heatmap_data.shape[1]],
                    y=list(heatmap_data.index),
                    colorscale='RdYlGn',
                    zmid=0
                ))
               
                heatmap.update_layout(
                    title='Average P&L by Hour and Day',
                    xaxis_title='Day of Week',
                    yaxis_title='Hour of Day',
                    template='plotly_dark',
                    height=400
                )
               
                heatmap_html = heatmap.to_html(include_plotlyjs=False, div_id="heatmap-plot")
                
                # Verbal description
                max_val = heatmap_data.max().max()
                min_val = heatmap_data.min().min()
                max_loc = heatmap_data.stack().idxmax()
                min_loc = heatmap_data.stack().idxmin()
                days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
                best_day = days[max_loc[1]] if max_loc[1] < len(days) else 'Unknown'
                best_hour = max_loc[0]
                worst_day = days[min_loc[1]] if min_loc[1] < len(days) else 'Unknown'
                worst_hour = min_loc[0]
                heatmap_description = f"""<p class="analysis-text">The heatmap shows average PnL by hour and day of week. 
The best performance is on {best_day} at {best_hour}:00 with average ${max_val:.2f}.
The worst is on {worst_day} at {worst_hour}:00 with ${min_val:.2f}.</p>"""
       
        # Note: Comparison chart has been moved to generate_comparison_chart method
       
        plots_html = f"""
        <div class="section">
            <h2>📊 Additional Performance Charts</h2>
            <div class="plot-container">
                {equity_html}
            </div>
            <div class="plot-container">
                {pnl_html}
            </div>
            {'<div class="plot-container">' + heatmap_html + heatmap_description + '</div>' if heatmap_html else ''}
        </div>
        """
       
        return plots_html
   
    @staticmethod
    def generate_ai_analysis(metrics, trades_df, df=None):
        """Generate AI-style analysis text"""
       
        # Performance classification
        if metrics.get('total_return_%', 0) > 20:
            perf_class = "exceptional"
        elif metrics.get('total_return_%', 0) > 10:
            perf_class = "strong"
        elif metrics.get('total_return_%', 0) > 0:
            perf_class = "positive"
        else:
            perf_class = "negative"
       
        # Risk assessment
        if metrics.get('max_drawdown_%', 0) > -10:
            risk_level = "low"
        elif metrics.get('max_drawdown_%', 0) > -20:
            risk_level = "moderate"
        else:
            risk_level = "high"
       
        # Strategy effectiveness
        if metrics.get('profit_factor', 0) > 2:
            strategy_effect = "highly effective"
        elif metrics.get('profit_factor', 0) > 1.5:
            strategy_effect = "effective"
        elif metrics.get('profit_factor', 0) > 1:
            strategy_effect = "marginally profitable"
        else:
            strategy_effect = "unprofitable"
       
        analysis = f"""
        <h3>📊 Performance Overview</h3>
        <p>The backtesting results show <strong>{perf_class} performance</strong> with a total return of
        <strong>{metrics.get('total_return_%', 0):.2f}%</strong>. The strategy executed
        <strong>{metrics.get('total_trades', 0)} trades</strong> with a win rate of
        <strong>{metrics.get('win_rate_%', 0):.1f}%</strong>, indicating
        {'consistent profitability' if metrics.get('win_rate_%', 0) > 50 else 'room for improvement'}.</p>
       
        <h3>💰 Profitability Analysis</h3>
        <p>The strategy proved to be <strong>{strategy_effect}</strong> with a profit factor of
        <strong>{metrics.get('profit_factor', 0):.2f}</strong>. The average winning trade generated
        <strong>${metrics.get('avg_win_$', 0):.2f}</strong> ({metrics.get('avg_win_pips', 0):.1f} pips),
        while the average losing trade lost <strong>${metrics.get('avg_loss_$', 0):.2f}</strong>
        ({metrics.get('avg_loss_pips', 0):.1f} pips).
        {'The favorable win/loss ratio suggests good trade selection.' if metrics.get('avg_win_$', 0) > metrics.get('avg_loss_$', 0) else 'Consider improving the risk/reward ratio.'}</p>
       
        <h3>⚠️ Risk Assessment</h3>
        <p>The strategy exhibited <strong>{risk_level} risk</strong> with a maximum drawdown of
        <strong>{metrics.get('max_drawdown_%', 0):.2f}%</strong>. The Sharpe ratio of
        <strong>{metrics.get('sharpe_ratio', 0):.2f}</strong>
        {'indicates good risk-adjusted returns' if metrics.get('sharpe_ratio', 0) > 1 else 'suggests the returns may not adequately compensate for the risk taken'}.
        The Sortino ratio of <strong>{metrics.get('sortino_ratio', 0):.2f}</strong>
        {'shows effective downside risk management' if metrics.get('sortino_ratio', 0) > 1 else 'indicates potential downside risk concerns'}.</p>
       
        <h3>🎯 Key Insights</h3>
        <ul>
            <li>Best performing trade: <strong>${metrics.get('best_trade_$', 0):.2f}</strong>
                ({metrics.get('best_trade_pips', 0):.1f} pips)</li>
            <li>Worst performing trade: <strong>${metrics.get('worst_trade_$', 0):.2f}</strong>
                ({metrics.get('worst_trade_pips', 0):.1f} pips)</li>
            <li>Average time in trade: <strong>{metrics.get('avg_time_in_trade_hours', 0):.1f} hours</strong></li>
            <li>Annualized return: <strong>{metrics.get('annualized_return_%', 0):.2f}%</strong></li>
        </ul>
       
        <h3>💡 Recommendations</h3>
        <p>{'The strategy shows promise and could benefit from position sizing optimization to maximize returns while managing risk.' if metrics.get('total_return_%', 0) > 0 else 'Consider revising entry/exit criteria and risk management parameters to improve performance.'}
        {'The high win rate suggests the entry signals are reliable.' if metrics.get('win_rate_%', 0) > 60 else ''}
        {'Focus on improving the average win/loss ratio through better profit targets or tighter stop losses.' if metrics.get('avg_win_$', 0) < metrics.get('avg_loss_$', 0) * 1.5 else ''}</p>
        """
       
        # Add comparison analysis if df provided
        if df is not None and not df.empty:
            bh_return = (df['close'].iloc[-1] - df['close'].iloc[0]) / df['close'].iloc[0] * 100 if not df.empty else 0
            strat_return = metrics.get('total_return_%', 0)
            if strat_return > bh_return:
                comp_text = f"The strategy outperformed the buy-and-hold return of {bh_return:.2f}% by achieving {strat_return:.2f}%."
                value_add = "adds value beyond passive holding"
            else:
                comp_text = f"The strategy underperformed compared to buy-and-hold return of {bh_return:.2f}%, achieving {strat_return:.2f}%."
                value_add = "may need adjustments to beat the market"
            analysis += f"""
            <h3>📈 Strategy vs Market Comparison</h3>
            <p>{comp_text} This suggests the strategy {value_add}.</p>
            """
       
        return analysis
# ======================================================================
# 7. ENHANCED UI WITH CALENDAR AND BETTER CONTROLS
# ======================================================================
class EnhancedBacktesterApp:
    def __init__(self, master):
        self.master = master
        master.title("AlgoHaus Backtester v4.0 - by GhostOkaamiii")
        master.configure(bg='#1e1e1e')
        master.option_add('*Font', 'Helvetica 9')
       
        # Default values
        self.base_folder_var = tk.StringVar(value=r"C:\Users\Wolfrank\Desktop\AlgoHaus\OandaHistoricalData\1MinCharts")
        self.pair_names = []
        self.pair_date_ranges = {}
       
        self.strategies = {
            "Opening Range": TradingStrategies.opening_range_strategy,
            "VWAP Crossover": TradingStrategies.vwap_crossover_strategy,
            "Pivot Reversal": TradingStrategies.pivot_point_reversal_strategy
        }
        self.timeframes = ['1min', '5min', '15min', '1hr', '1Day']
       
        # Trading parameters
        self.strategy_var = tk.StringVar(value="Opening Range")
        self.pair_var = tk.StringVar()
        self.timeframe_var = tk.StringVar(value='1min')
        self.start_date_var = tk.StringVar(value="2025-01-01")
        self.end_date_var = tk.StringVar(value="2025-10-05") # Default end date
        self.sl_var = tk.StringVar(value="50")
        self.tp_var = tk.StringVar(value="100")
       
        # Risk management
        self.initial_balance_var = tk.StringVar(value="10000")
        self.unit_size_var = tk.StringVar(value="10000")
        self.leverage_var = tk.StringVar(value="50")
        self.risk_percent_var = tk.StringVar(value="2")
       
        # Status
        self.status_var = tk.StringVar(value="Ready")
        self.loaded_df = None
        self.last_bt = None
        self.last_metrics = None
        self.csv_date_range = None
        self.current_pair_range = None
       
        self.task_queue = queue.Queue()
        self.setup_style()
        self.create_widgets()
        self.refresh_pairs()
        self.master.after(100, self.process_queue)
   
    def setup_style(self):
        """Setup dark theme styling"""
        s = ttk.Style()
        s.theme_create("Dark", parent="alt", settings={
            ".": {"configure": {"background": "#1e1e1e", "foreground": "#e0e0e0"}},
            "TFrame": {"configure": {"background": "#1e1e1e"}},
            "TLabel": {"configure": {"background": "#1e1e1e", "foreground": "#cccccc"}},
            "TButton": {"configure": {"padding": [6, 3], "font": "Helvetica 8 bold"}},
            "Blue.TButton": {"configure": {"background": "#1976D2", "foreground": "white"}},
            "TEntry": {"configure": {"fieldbackground": "#2d2d2d", "foreground": "#e0e0e0"}},
            "TCombobox": {"configure": {"fieldbackground": "#2d2d2d", "foreground": "#e0e0e0"}}
        })
        s.theme_use("Dark")
   
    def create_widgets(self):
        """Create the UI components"""
        # Header
        header = ttk.Frame(self.master, padding="10 8")
        header.pack(fill='x')
        ttk.Label(header, text="AlgoHaus Backtester v4.0", font=('Helvetica', 12, 'bold')).pack(side='left')
        ttk.Label(header, text="by GhostOkaamiii (Wolfrank Guzman)", font=('Helvetica', 9, 'italic')).pack(side='left', padx=(20, 0))
       
        # Main container
        main = ttk.Frame(self.master, padding=12)
        main.pack(fill='both', expand=True)
       
        # Left panel
        left = ttk.Frame(main, relief='ridge', borderwidth=1, padding=10)
        left.grid(row=0, column=0, sticky='nsew', padx=5, pady=5)
       
        row = 0
       
        # === DATA SECTION ===
        ttk.Label(left, text="DATA SOURCE", font=('Helvetica', 10, 'bold')).grid(row=row, column=0, columnspan=3, sticky='w', pady=(0,5))
        row += 1
       
        # Base Folder
        ttk.Label(left, text="Base Folder:").grid(row=row, column=0, sticky='w', pady=2)
        ttk.Entry(left, textvariable=self.base_folder_var, width=40).grid(row=row, column=1, columnspan=2, sticky='ew', pady=2, padx=5)
        row += 1
       
        # Pair selection
        ttk.Label(left, text="Pair:").grid(row=row, column=0, sticky='w', pady=2)
        self.pair_combo = ttk.Combobox(left, textvariable=self.pair_var, state="readonly", width=15)
        self.pair_combo.grid(row=row, column=1, sticky='ew', pady=2, padx=5)
        ttk.Button(left, text="Refresh Pairs", command=self.refresh_pairs).grid(row=row, column=2, padx=5)
        row += 1
       
        # Pair info display
        self.pair_info_text = tk.Text(left, height=3, width=40, bg='#2d2d2d', fg='#e0e0e0', font=('Consolas', 8))
        self.pair_info_text.grid(row=row, column=0, columnspan=3, sticky='ew', pady=5)
        row += 1
       
        # === DATE RANGE SECTION ===
        ttk.Label(left, text="DATE RANGE", font=('Helvetica', 10, 'bold')).grid(row=row, column=0, columnspan=3, sticky='w', pady=(10,5))
        row += 1
       
        # Date inputs with calendar
        date_frame = ttk.Frame(left)
        date_frame.grid(row=row, column=0, columnspan=3, sticky='ew', pady=3)
       
        ttk.Label(date_frame, text="Start:").pack(side='left')
        self.start_date_entry = ttk.Entry(date_frame, textvariable=self.start_date_var, width=12)
        self.start_date_entry.pack(side='left', padx=2)
       
        ttk.Label(date_frame, text="End:").pack(side='left', padx=(10,0))
        self.end_date_entry = ttk.Entry(date_frame, textvariable=self.end_date_var, width=12)
        self.end_date_entry.pack(side='left', padx=2)
       
        ttk.Button(date_frame, text="AUTO", command=self.set_auto_date, style='Blue.TButton').pack(side='left', padx=5)
        row += 1
       
        # Quick date ranges
        range_frame = ttk.Frame(left)
        range_frame.grid(row=row, column=0, columnspan=3, sticky='ew', pady=2)
        for txt, days in [("30d", 30), ("90d", 90), ("180d", 180), ("1Y", 365), ("2Y", 730), ("5Y", 1825)]:
            ttk.Button(range_frame, text=txt, command=lambda d=days: self.set_date_range(d)).pack(side='left', padx=2)
        row += 1
       
        # === STRATEGY SECTION ===
        ttk.Label(left, text="STRATEGY", font=('Helvetica', 10, 'bold')).grid(row=row, column=0, columnspan=3, sticky='w', pady=(10,5))
        row += 1
       
        # Strategy selection
        ttk.Label(left, text="Strategy:").grid(row=row, column=0, sticky='w', pady=2)
        ttk.Combobox(left, textvariable=self.strategy_var, values=list(self.strategies), state="readonly", width=20).grid(row=row, column=1, columnspan=2, sticky='ew', pady=2, padx=5)
        row += 1
       
        # Timeframe
        ttk.Label(left, text="Timeframe:").grid(row=row, column=0, sticky='w', pady=2)
        ttk.Combobox(left, textvariable=self.timeframe_var, values=self.timeframes, state="readonly", width=20).grid(row=row, column=1, columnspan=2, sticky='ew', pady=2, padx=5)
        row += 1
       
        # SL/TP
        sl_tp_frame = ttk.Frame(left)
        sl_tp_frame.grid(row=row, column=0, columnspan=3, sticky='w', pady=2)
        ttk.Label(sl_tp_frame, text="SL (pips):").pack(side='left')
        ttk.Entry(sl_tp_frame, textvariable=self.sl_var, width=8).pack(side='left', padx=2)
        ttk.Label(sl_tp_frame, text="TP (pips):").pack(side='left', padx=(15,0))
        ttk.Entry(sl_tp_frame, textvariable=self.tp_var, width=8).pack(side='left', padx=2)
        row += 1
       
        # === RISK MANAGEMENT SECTION ===
        ttk.Label(left, text="RISK MANAGEMENT", font=('Helvetica', 10, 'bold')).grid(row=row, column=0, columnspan=3, sticky='w', pady=(10,5))
        row += 1
       
        # Initial Balance
        ttk.Label(left, text="Initial Balance ($):").grid(row=row, column=0, sticky='w', pady=2)
        ttk.Entry(left, textvariable=self.initial_balance_var, width=15).grid(row=row, column=1, sticky='w', pady=2, padx=5)
        row += 1
       
        # Unit Size
        ttk.Label(left, text="Unit Size:").grid(row=row, column=0, sticky='w', pady=2)
        ttk.Entry(left, textvariable=self.unit_size_var, width=15).grid(row=row, column=1, sticky='w', pady=2, padx=5)
        ttk.Button(left, text="Calculate", command=self.calculate_position_size).grid(row=row, column=2, padx=5)
        row += 1
       
        # Leverage
        ttk.Label(left, text="Leverage:").grid(row=row, column=0, sticky='w', pady=2)
        leverage_combo = ttk.Combobox(left, textvariable=self.leverage_var,
                                      values=ForexCalculator.LEVERAGE_OPTIONS,
                                      state="readonly", width=10)
        leverage_combo.grid(row=row, column=1, sticky='w', pady=2, padx=5)
        row += 1
       
        # Risk %
        ttk.Label(left, text="Risk per Trade (%):").grid(row=row, column=0, sticky='w', pady=2)
        ttk.Entry(left, textvariable=self.risk_percent_var, width=10).grid(row=row, column=1, sticky='w', pady=2, padx=5)
        row += 1
       
        # Status
        ttk.Label(left, text="Status:").grid(row=row, column=0, sticky='w', pady=5)
        ttk.Label(left, textvariable=self.status_var, foreground='#1976D2').grid(row=row, column=1, columnspan=2, sticky='w', pady=5, padx=5)
        row += 1
       
        # Action buttons
        button_frame = ttk.Frame(left)
        button_frame.grid(row=row, column=0, columnspan=3, pady=10)
       
        ttk.Button(button_frame, text="LOAD DATA", command=lambda: self.run_async(self.load_async), style='Blue.TButton').pack(side='left', padx=2)
        ttk.Button(button_frame, text="RUN BACKTEST", command=lambda: self.run_async(self.backtest_async), style='Blue.TButton').pack(side='left', padx=2)
        ttk.Button(button_frame, text="GENERATE REPORT", command=self.gen_report, style='Blue.TButton').pack(side='left', padx=2)
       
        left.grid_columnconfigure(1, weight=1)
       
        # Right panel - Results
        right = ttk.Frame(main, relief='ridge', borderwidth=1, padding=10)
        right.grid(row=0, column=1, sticky='nsew', padx=5, pady=5)
       
        ttk.Label(right, text="Results", font=('Helvetica', 11, 'bold')).pack(anchor='w', pady=(0,10))
       
        self.results = scrolledtext.ScrolledText(right, bg='#2d2d2d', fg='#e0e0e0', font=('Consolas', 9))
        self.results.pack(fill='both', expand=True)
       
        main.grid_columnconfigure(1, weight=1)
        main.grid_rowconfigure(0, weight=1)
       
        # Bind pair change event
        self.pair_var.trace('w', self.on_pair_change)
   
    def process_queue(self):
        """Process messages from async operations"""
        try:
            while True:
                msg = self.task_queue.get_nowait()
                if msg['type'] == 'status':
                    self.status_var.set(msg['text'])
                elif msg['type'] == 'result':
                    self.display_message(msg['text'])
                elif msg['type'] == 'error':
                    messagebox.showerror("Error", msg['text'])
        except queue.Empty:
            pass
        self.master.after(100, self.process_queue)
   
    def refresh_pairs(self):
        """Refresh the list of available pairs"""
        try:
            base = pathlib.Path(self.base_folder_var.get())
            if not base.exists():
                return
           
            folders = []
            for item in base.iterdir():
                if item.is_dir():
                    pair_name = item.name.replace('_', '/')
                    if '/' in pair_name and any(curr in pair_name for curr in ['USD', 'EUR', 'GBP', 'JPY', 'CAD', 'AUD', 'NZD', 'CHF']):
                        folders.append(pair_name)
           
            self.pair_names = sorted(folders)
            self.pair_combo['values'] = self.pair_names
           
            if self.pair_names:
                self.pair_var.set(self.pair_names[0])
        except Exception as e:
            print(f"Error refreshing pairs: {e}")
   
    def on_pair_change(self, *args):
        """Update pair information display"""
        pair = self.pair_var.get()
        if not pair:
            return
       
        # Update pair info
        pip_value = ForexCalculator.PIP_VALUES.get(pair, 0.0001)
        unit_size = int(self.unit_size_var.get() or 10000)
        leverage = int(self.leverage_var.get() or 50)
       
        # Assume a typical price for margin calculation
        typical_price = 1.0 if pair.endswith('/USD') else 100.0 if 'JPY' in pair else 1.2
       
        pip_value_usd = ForexCalculator.calculate_pip_value_in_usd(pair, unit_size, typical_price)
        margin_req = ForexCalculator.calculate_margin_required(pair, unit_size, typical_price, leverage)
       
        info_text = f"""Pair: {pair}
Pip Value (${unit_size:,} units): ${pip_value_usd:.4f}
Margin Required: ${margin_req:.2f} (Leverage: {leverage}:1)"""
       
        self.pair_info_text.delete(1.0, tk.END)
        self.pair_info_text.insert(tk.END, info_text)
   
    def calculate_position_size(self):
        """Calculate optimal position size based on risk"""
        try:
            balance = float(self.initial_balance_var.get())
            risk_pct = float(self.risk_percent_var.get())
            sl_pips = float(self.sl_var.get())
            pair = self.pair_var.get()
           
            if not pair:
                return
           
            # Use typical price
            typical_price = 1.0 if pair.endswith('/USD') else 100.0 if 'JPY' in pair else 1.2
           
            position_size = ForexCalculator.calculate_position_size(
                balance, risk_pct, sl_pips, pair, typical_price
            )
           
            self.unit_size_var.set(str(position_size))
            self.on_pair_change() # Update display
           
        except ValueError:
            messagebox.showwarning("Input Error", "Please enter valid numeric values")
   
    def set_auto_date(self):
        """Set date range to maximum available"""
        self.start_date_var.set("2020-01-01")
        self.end_date_var.set("2025-10-05")
   
    def set_date_range(self, days_back):
        """Set date range for specified days back"""
        end_date = datetime.strptime("2025-10-05", "%Y-%m-%d")
        start_date = end_date - timedelta(days=days_back)
        self.start_date_var.set(start_date.strftime("%Y-%m-%d"))
        self.end_date_var.set(end_date.strftime("%Y-%m-%d"))
   
    def run_async(self, func):
        """Run function asynchronously"""
        self.status_var.set("Processing...")
        threading.Thread(target=func, daemon=True).start()
   
    def load_async(self):
        """Load data asynchronously"""
        def worker():
            try:
                pair = self.pair_var.get()
                if not pair:
                    raise ValueError("Select a pair")
               
                start = datetime.strptime(self.start_date_var.get(), "%Y-%m-%d")
                end = datetime.strptime(self.end_date_var.get(), "%Y-%m-%d")
                base = pathlib.Path(self.base_folder_var.get())
                tf = self.timeframe_var.get()
               
                self.task_queue.put({'type': 'status', 'text': 'Loading data...'})
               
                df, actual_start, actual_end = load_pair_data(pair, base, start, end, tf)
                self.csv_date_range = (actual_start, actual_end)
                self.loaded_df = df
               
                bars = len(df)
                memory_mb = df.memory_usage(deep=True).sum() / 1024 / 1024
               
                self.task_queue.put({'type': 'status', 'text': f"Loaded {bars:,} bars"})
               
                summary = (f"Data Loaded Successfully\n"
                          f"Pair: {pair}\n"
                          f"Timeframe: {tf}\n"
                          f"Bars: {bars:,}\n"
                          f"Date Range: {df['datetime'].min().date()} to {df['datetime'].max().date()}\n"
                          f"Memory: {memory_mb:.1f} MB")
               
                self.task_queue.put({'type': 'result', 'text': summary})
               
            except Exception as e:
                self.task_queue.put({'type': 'error', 'text': str(e)})
                self.task_queue.put({'type': 'status', 'text': 'Load failed'})
       
        threading.Thread(target=worker, daemon=True).start()
   
    def backtest_async(self):
        """Run backtest asynchronously"""
        if not hasattr(self, 'loaded_df') or self.loaded_df is None:
            self.task_queue.put({'type': 'error', 'text': "Load data first"})
            return
       
        def worker():
            try:
                # Get parameters
                sl = float(self.sl_var.get())
                tp = float(self.tp_var.get())
                initial_balance = float(self.initial_balance_var.get())
                unit_size = int(self.unit_size_var.get())
                leverage = int(self.leverage_var.get())
                pair = self.pair_var.get()
                pip_value = ForexCalculator.PIP_VALUES.get(pair, 0.0001)
                strat = self.strategies[self.strategy_var.get()]
               
                # Create backtester
                bt = EnhancedBacktester(
                    self.loaded_df,
                    initial_balance=initial_balance,
                    unit_size=unit_size,
                    pip_value=pip_value,
                    leverage=leverage
                )
               
                self.last_bt = bt
                self.task_queue.put({'type': 'status', 'text': 'Running backtest...'})
               
                # Run backtest
                import time
                start_time = time.time()
                summary, metrics = bt.run_backtest(strat, sl, tp, pair)
                elapsed = time.time() - start_time
               
                self.last_metrics = metrics
               
                # Format results with more details
                result_text = (f"BACKTEST COMPLETE (in {elapsed:.2f} seconds)\n"
                              f"{summary}\n\n"
                              f"ALL METRICS:\n")
                for key, value in metrics.items():
                    result_text += f"{key.replace('_', ' ').title()}: {value}\n"
               
                self.task_queue.put({'type': 'result', 'text': result_text})
                self.task_queue.put({'type': 'status', 'text': f'Complete - {len(bt.results)} trades'})
               
            except Exception as e:
                self.task_queue.put({'type': 'error', 'text': f"Backtest failed: {str(e)}"})
                self.task_queue.put({'type': 'status', 'text': 'Backtest failed'})
       
        threading.Thread(target=worker, daemon=True).start()
   
    def gen_report(self):
        """Generate and save HTML report"""
        if not self.last_bt or self.last_bt.results is None or self.last_bt.results.empty:
            messagebox.showwarning("No Data", "Run backtest first")
            return
       
        try:
            # Generate report
            html, timestamp = AdvancedHTMLReportGenerator.generate_dashboard_report(
                self.last_metrics,
                self.last_bt.results,
                self.strategy_var.get(),
                self.timeframe_var.get(),
                self.pair_var.get(),
                float(self.initial_balance_var.get()),
                int(self.unit_size_var.get()),
                df=self.loaded_df
            )
           
            # Create filename
            pair_clean = self.pair_var.get().replace('/', '')
            strategy_clean = self.strategy_var.get().replace(' ', '_')
            filename = f"backtest_{pair_clean}_{strategy_clean}_{timestamp}.html"
           
            # Save to multiple locations
            save_paths = [
                pathlib.Path.home() / "Desktop" / filename,
                pathlib.Path("guzmanwolfrank/test/Quantstuff/oanda25/backtest/backtest_results") / filename
            ]
           
            saved_locations = []
            for save_path in save_paths:
                try:
                    save_path.parent.mkdir(parents=True, exist_ok=True)
                    with open(save_path, 'w', encoding='utf-8') as f:
                        f.write(html)
                    saved_locations.append(str(save_path))
                except Exception as e:
                    print(f"Could not save to {save_path}: {e}")
           
            # Also save to temp and open
            import tempfile
            with tempfile.NamedTemporaryFile('w', suffix='.html', delete=False, encoding='utf-8') as f:
                f.write(html)
                temp_path = f.name
           
            webbrowser.open('file://' + temp_path)
           
            # Show success message
            msg = "Report saved to:\n" + "\n".join(saved_locations) if saved_locations else "Report opened in browser"
            self.display_message(msg)
           
        except Exception as e:
            messagebox.showerror("Error", f"Failed to generate report: {str(e)}")
   
    def display_message(self, text):
        """Display message in results area"""
        self.results.delete(1.0, tk.END)
        self.results.insert(tk.END, text)
# ======================================================================
# 8. MAIN ENTRY POINT
# ======================================================================
if __name__ == '__main__':
    try:
        # Check for required packages
        import plotly
    except ImportError:
        print("Installing required packages...")
        import subprocess
        subprocess.check_call(["pip", "install", "plotly", "pandas", "numpy"])
   
    root = tk.Tk()
    app = EnhancedBacktesterApp(root)
    root.geometry("1200x800")
    root.mainloop()

c:\Users\Wolfrank\AppData\Local\Programs\Python\Python311\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

C:\Users\Wolfrank\AppData\Local\Temp\ipykernel_18656\1215228165.py:232: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.

c:\Users\Wolfrank\AppData\Local\Programs\Python\Python311\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

